# Aries Basic Controller - Openmined PryVote Community

This notebook walks through a MVP for PryVote assuming the Openminded community will be issued an Openmined commuity credential that will enable community members to vote on basic things. The credential will be issued across a previously established connection. 

It is best run in parallel with the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) controlling the agent receiving the credential to enable voting.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://red-agent:8051"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## 1. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)


## 3. Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Red and Blue**


In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'routing_state': 'none', 'accept': 'manual', 'initiator': 'external', 'invitation_mode': 'once', 'my_did': 'LXJpbiS6f868VepB5PC15r', 'their_did': 'Ejnten3EaJZScUkQtGvq1K', 'state': 'active', 'their_label': 'Bob', 'request_id': '23270601-a0f1-45e4-b13b-8f623411539b', 'created_at': '2020-09-09 18:10:38.537889Z', 'invitation_key': '3EvYNwatLFnWE63ZCF5bdfzSnVVHBekSvN55s1ug8Lbh', 'updated_at': '2020-09-09 18:10:50.799442Z', 'connection_id': '701c408c-be28-4f90-9c6a-7c1e56f7240d'}]
Connection : {'routing_state': 'none', 'accept': 'manual', 'initiator': 'external', 'invitation_mode': 'once', 'my_did': 'LXJpbiS6f868VepB5PC15r', 'their_did': 'Ejnten3EaJZScUkQtGvq1K', 'state': 'active', 'their_label': 'Bob', 'request_id': '23270601-a0f1-45e4-b13b-8f623411539b', 'created_at': '2020-09-09 18:10:38.537889Z', 'invitation_key': '3EvYNwatLFnWE63ZCF5bdfzSnVVHBekSvN55s1ug8Lbh', 'updated_at': '2020-09-09 18:10:50.799442Z', 'connection_id': '701c408c-be28-4f90-9c6a-7c1e56f7240d'}
Active Conne

## 4. Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [6]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_community-member1"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "community_member"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1


## 5. Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [7]:
response = await agent_controller.definitions.write_cred_def(schema_id)
print(response)
cred_def_id = response["credential_definition_id"]
print(cred_def_id)

{'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default'}
PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


**Note: You should be able to see both schema and definition transactions on the local network [here](http://localhost:9000)**

## 6. Populate the Attribues to Issue to Openmind-pryvote-voter (Bob)

The schema defines two attributes: name and community_member. Both must be populated with attribute values that Openmined Community (Alice) wishes to issue in a credential to Openmind-pryvote-voter (Bob). To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [8]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "community_member", "value": "1"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## 7. Send Credential

This sends a credential to a holder Openmind-pryvote-voter (Bob), and automates the rest of the protocol. This tutorial works best if you have initialised the [openmind-pryvote-voter notebook](http://127.0.0.1:8889/notebooks/openmind-pryvote-voter.ipynb) and registered the listener for the issue_credential webhook.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [9]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
#extract issuer DID for proof request requirements
issuer_did = record['credential_proposal_dict']['issuer_did']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]}, 'connection_id': '701c408c-be28-4f90-9c6a-7c1e56f7240d', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'schema_name': 'open_mined_community-member1', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Credential exchange 6dafaafe-fddd-4e08-abed-746892b546d6, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 6dafaafe-fddd-4e08-abed-746892b546d6, role: issuer, state: offer_sent
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


## 9. Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [10]:
response = await agent_controller.issuer.get_records()
print(response['results'])

[{'auto_remove': True, 'credential_definition_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '19984204381732266376363148717959803890458327717202459028255648158811158974658', 'xz_cap': '404476306924503912581862354234357544712825268065583322461657768393257962009373120730592257730695921390547306954743265906635006026480691910885148106111608858368064003936306678243752869137358968948552786617284160075599165356927499347819191998684194788058368166778606671090535457935858977995871942183249140602946405117033855729886377030257279115204034022104334625684884653727893406420356073500710239893050249019867001405901814676804509961097080322650645110442843948415787179095766023163539686790052855221164260177590151357596773748377784432529012770930498499537490709642343513885932895181953400100712708859044830136423421455596373269

In [11]:
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 6dafaafe-fddd-4e08-abed-746892b546d6, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 6dafaafe-fddd-4e08-abed-746892b546d6, role: issuer, state: request_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange 6dafaafe-fddd-4e08-abed-746892b546d6, role: issuer, state: credential_issued
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange 6dafaafe-fddd-4e08-abed-746892b546d6, role: issuer, state: credential_acked
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


# 12. Send Proof Request

Request a proof of Opemined community member to prove community membership

In [15]:
print("Request proof of Openmined Community from Openmined PryVote Voter (Bob)")
#Set some variables

revocation = False
SELF_ATTESTED = True
exchange_tracing = False

#Enable this to ask for attributes to identity a user
req_attrs = [
    {"name": "name", "restrictions": [{"issuer_did": issuer_did}]},
]

if SELF_ATTESTED:
    # test self-attested claims
    req_attrs.append({"name": "community_vote"},)
    
#Set predicates for Zero Knowledge Proofs
req_preds = [
    # test zero-knowledge proofs
    {
        "name": "community_member",
        "p_type": ">=",
        "p_value": 1,
        "restrictions": [{"issuer_did": issuer_did}],
    }
]

indy_proof_request = {
    "name": "Proof of Openmined Community Member",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid": 
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
        f"0_{req_pred['name']}_GE_uuid": 
        req_pred for req_pred in req_preds
    },
}

if revocation:
    indy_proof_request["non_revoked"] = {"to": int(time.time())}
    
proof_request = indy_proof_request

Request proof of Openmined Community from Openmined PryVote Voter (Bob)


In [22]:
request_body = {
    "connection_id": connection_id,
    "proof_request": proof_request,
    "comment": "",
    "trace": exchange_tracing
}

response = await agent_controller.proofs.send_request(request_body)
print(response)
presentation_exchange_id = response['presentation_exchange_id']
print("\n")
print(presentation_exchange_id)


{'role': 'verifier', 'auto_present': False, 'trace': False, 'presentation_exchange_id': 'bf52acd4-c09d-46d0-ba89-2af52615cb49', 'state': 'request_sent', 'connection_id': '701c408c-be28-4f90-9c6a-7c1e56f7240d', 'presentation_request': {'name': 'Proof of Openmined Community Member', 'version': '1.0', 'requested_attributes': {'0_name_uuid': {'name': 'name', 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}, '0_community_vote_uuid': {'name': 'community_vote'}}, 'requested_predicates': {'0_community_member_GE_uuid': {'name': 'community_member', 'p_type': '>=', 'p_value': 1, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '69315844992805527710604'}, 'created_at': '2020-09-09 18:45:43.785536Z', 'thread_id': '61b46e4d-fe2e-4e41-97de-5d4bb3ba4cbb', 'initiator': 'self', 'updated_at': '2020-09-09 18:45:43.785536Z'}


bf52acd4-c09d-46d0-ba89-2af52615cb49


# 16. Verify Proof Presentation

In [23]:
verify = await agent_controller.proofs.verify_presentation(presentation_exchange_id)
print(verify)

ClientResponseError: 500, message='Internal Server Error', url=URL('http://alice-agent:8021/present-proof/records/bf52acd4-c09d-46d0-ba89-2af52615cb49/verify-presentation')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/helpers/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/helpers/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/helpers/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

There are a few api calls not covered in this tutorial so far. Mostly around credential revocation.

In [24]:
response = await agent_controller.terminate()
print(response)

None
